# A simple tutorial pipleline for paramterization of M dwarf stars  

Jiadong LI (mailto:jdli at nao.cas.cn),  
Created on Fri. Mar 13 2020  
Revised on Thu. Oct 21 2021

In [1]:
import numpy as np
from tqdm import tqdm
from astropy.io import fits
from astropy.table import Table
from slam import Slam
from slam.normalization import normalize_spectrum, normalize_spectra_block
from preprocess import LAMOST

In [2]:
# specify a wavelength range from LAMOST DR5 (R~1800) spectra as useful pixels

wave_interp = np.load("models/wave_interp_R1800.npz")['wave']

# Preprocess: load all spectra data
obs_dir = "data/"
obs_spec = LAMOST(root_dir=obs_dir, wave_resamp=wave_interp)


fluxes, invars = [], []
for i in tqdm(range(len(obs_spec))):
    fluxes += [obs_spec[i]['flux_resamp']]
    invars += [obs_spec[i]['invar_resamp']]
    
fluxes, invars = np.array(fluxes), np.array(invars)

100%|██████████| 3/3 [00:00<00:00, 10.94it/s]


In [4]:
# normalization of each spetra

fluxes_norm, fluxes_cont = normalize_spectra_block(wave_interp, fluxes, 
                                                   (6147., 8910.), 10., p=(1E-8, 1E-7), q=0.7, 
                                                   eps=1E-19, rsv_frac=2., n_jobs=2, verbose=5)

invars_norm = fluxes_cont**2*invars

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   3 out of   3 | elapsed:    0.2s finished


In [5]:
# load SLAM model trained by APOGEE DR16-labeled spectra

k_ap = Slam.load_dump("models/ASPCAP_DR16.dump")

/Users/jordan/anaconda3/lib/python3.7/site-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator StandardScaler from version 0.22.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
/Users/jordan/anaconda3/lib/python3.7/site-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator MLPRegressor from version 0.22.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [6]:
label_init = k_ap.predict_labels_quick(fluxes_norm, invars_norm, n_jobs=2)

results_pred = k_ap.predict_labels_multi(label_init, fluxes_norm, invars_norm)
label_pred = np.array([label['x'] for label in results_pred])

TEFF_AP = label_pred[:,0]
MOH_AP  = label_pred[:,1]

@Slam.heal_the_world: IVAR of 0 spectra need to be reset


@Slam.heal_the_world: FLUX of 0 spectra need to be reset

@Slam.heal_the_world: IVAR of 0 spectra need to be reset


@Slam.heal_the_world: FLUX of 0 spectra need to be reset

@SLAM: The spectra with fewest pixels unmasked is [1161/1596]
@SLAM3: nfev=73, status=2, pstd=[1.24343613e+01 2.28025837e-02 2.45668509e-02 5.13824305e-03]
@SLAM3: nfev=248, status=2, pstd=[5.13475402e+00 1.95973222e-02 1.86161253e-02 3.22743998e-03]


/Users/jordan/anaconda3/lib/python3.7/site-packages/slam/predict.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ys = np.array(ys).T
/Users/jordan/anaconda3/lib/python3.7/site-packages/slam/predict.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ys = np.array(ys).T
/Users/jordan/anaconda3/lib/python3.7/site-packages/slam/predict.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, 

@SLAM3: nfev=86, status=2, pstd=[1.15654436e+01 3.02670937e-02 3.10535295e-02 5.95889171e-03]


/Users/jordan/anaconda3/lib/python3.7/site-packages/slam/predict.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ys = np.array(ys).T
/Users/jordan/anaconda3/lib/python3.7/site-packages/slam/predict.py:101: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ys = np.array(ys).T
